# Módulo 2 – Limpieza y Transformación de Datos

En esta sesión aprenderás a preparar tus datos antes de un análisis profundo. Trabajaremos con los datasets **`salarios.csv`** y **`paises.csv`** ubicados en `data/raw/`.


##  Objetivos de la sesión
- Detectar y manejar **valores nulos** y **duplicados**.
- Limpiar columnas monetarias y convertir tipos de datos.
- **Unir** (merge) datasets por claves comunes.
- Crear **nuevas columnas** útiles para el análisis.
- Exportar los datos limpios a `data/processed/`.


## Recuerden la Estructura de un proyecto

Una estructura recomendada puede ser:
```
project-name/
├── assest/           # Imagenes, graficos (generalmente no se sube a github)
├── data/             # Datos crudos y procesados
    ├──── raw/   
    ├──── processed/   --> Crear esta carpeta si no la tienen
├── notebooks/        # Notebooks del proyecto
├── utils/            # Funciones auxiliares
├── docs/             # Documentación
├── README.md         # Descripción general
├── environment.yml   # Dependencias usando conda
└── requirements.txt  # Dependencias usando pip

---
## 1 · Importar librerías y cargar datasets
Comenzamos cargando Pandas y leyendo los archivos CSV desde la carpeta `data/raw/`.


### 📦 Librerías que usaremos
Importamos módulos clave para:

* **pandas / numpy** → manipulación numérica y tabular  
* **matplotlib / seaborn** → gráficos rápidos (los usaremos más adelante)  
* **pathlib** → manejar rutas de forma independiente del sistema operativo

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

### 🔍 Ubicar el archivo de salarios
Utilizamos la constante `DATA_RAW_DIR` (definida en `utils.paths`) para construir
la ruta absoluta a **salarios.csv**.  
Esto evita escribir paths “hard-codeados” y hace el notebook portable.

In [2]:
!pwd

/Users/jssdev/Dev/Projects/Pydata/pydatapanama-cursos


In [2]:
#Importar la carpeta de datos raw desde paths
from cursos.analisis_datos.utils.paths import DATA_RAW_DIR

#definir un path o ruta con Path
path_salarios = DATA_RAW_DIR / "salarios.csv"
path_salarios = str(path_salarios)

print(f"Ruta del archivo salarios: {path_salarios}")
print(f"existe archivo salarios: {Path(path_salarios).exists()}")

Ruta del archivo salarios: /Users/jssdev/Dev/Projects/Pydata/pydatapanama-cursos/cursos/analisis_datos/data/raw/salarios.csv
existe archivo salarios: True


### 📥 Carga del dataset de salarios
Leemos el archivo con `pd.read_csv()`.  
* `sep=','` porque es un CSV estándar.  
* `encoding='utf-8'` asegura que caracteres latinos/acentos se lean bien.

In [3]:
df_salarios = pd.read_csv(filepath_or_buffer=path_salarios, sep=',', encoding='utf-8')

In [4]:
df_salarios.head(7)

,nombre,apellido,edad,pais,educacion_universitaria,nombre_de_la_empresa,cargo,anos_en_la_empresa,sueldo_anual,bono_anual
0,José,López,36,Chile,Sí,Mercado Libre,Python Developer,2,$71841,€4981
1,Fernanda,Vargas,39,Chile,Sí,Amazon,AI Software Engineer,2,$142115,€1252
2,Fernanda,Ramírez,30,Colombia,No,Facebook,Machine Learning Engineer,2,$92278,€5262
3,José,Gómez,35,Chile,Sí,Mercado Libre,Python Developer,12,$97130,€3671
4,Ana,Pérez,32,Panamá,No,Amazon,Project Manager,2,$61413,€10190
5,Andrea,Torres,45,Costa Rica,No,Mercado Libre,Data Engineer,8,$84987,€5493
6,Andrea,Vargas,58,Chile,No,Mercado Libre,Data Scientist,9,$96936,€4911


---
## 2 · Explorar la estructura del dataset
### 2.1 Información general
Revisamos tipos de datos, cantidad de filas/columnas y valores nulos.


### 🏷️ Inspección rápida del DataFrame
`df.info()` resume:
* Columnas y tipos (`object`, `int64`, etc.)  
* Filas no-nulas por columna → detectamos valores faltantes (`pais`, `empresa`)  
* Memoria ocupada

In [5]:
df_salarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   nombre                   325 non-null    object
 1   apellido                 325 non-null    object
 2   edad                     325 non-null    int64 
 3   pais                     324 non-null    object
 4   educacion_universitaria  325 non-null    object
 5   nombre_de_la_empresa     317 non-null    object
 6   cargo                    325 non-null    object
 7   anos_en_la_empresa       325 non-null    int64 
 8   sueldo_anual             325 non-null    object
 9   bono_anual               325 non-null    object
dtypes: int64(2), object(8)
memory usage: 25.5+ KB


### 2.2 Estadísticas descriptivas rápidas


### 📊 Estadísticas descriptivas
`df.describe()` genera medidas (count, mean, std…) solo para columnas
numéricas. Aquí vemos *edad* y *años en la empresa*.  
Esto ayuda a detectar outliers y entender la dispersión.

In [6]:
df_salarios.describe()

,edad,anos_en_la_empresa
count,325.000000,325.000000
mean,40.889231,6.353846
std,11.582769,3.997803
min,22.000000,0.000000
25%,30.000000,3.000000
50%,41.000000,6.000000
75%,51.000000,10.000000
max,60.000000,13.000000


---
## 3 · Limpiar columnas monetarias
Las columnas `sueldo_anual` y `bono_anual` tienen símbolos de moneda. Eliminamos `$` y `€` y convertimos a `float`.


In [7]:
df_salarios.head()

,nombre,apellido,edad,pais,educacion_universitaria,nombre_de_la_empresa,cargo,anos_en_la_empresa,sueldo_anual,bono_anual
0,José,López,36,Chile,Sí,Mercado Libre,Python Developer,2,$71841,€4981
1,Fernanda,Vargas,39,Chile,Sí,Amazon,AI Software Engineer,2,$142115,€1252
2,Fernanda,Ramírez,30,Colombia,No,Facebook,Machine Learning Engineer,2,$92278,€5262
3,José,Gómez,35,Chile,Sí,Mercado Libre,Python Developer,12,$97130,€3671
4,Ana,Pérez,32,Panamá,No,Amazon,Project Manager,2,$61413,€10190


Revisar las columnas `sueldo_anual` y `bono_anual` pues tienen los simbolos de moneda

### 💲 Columnas monetarias originales
Previsualizamos `sueldo_anual` y `bono_anual` antes de limpiarlas.
Observa que contienen símbolos (`$`, `€`) 

In [8]:
# escribe tu codigo aqui
df_salarios[['sueldo_anual', 'bono_anual']]

,sueldo_anual,bono_anual
0,$71841,€4981
1,$142115,€1252
2,$92278,€5262
3,$97130,€3671
4,$61413,€10190
...,...,...
320,$97114,€1684
321,$104541,€3128
322,$86042,€8746
323,$96078,€11419


Renombrar las columnas usando `.rename`

```python
columns={
        "sueldo_anual": "sueldo_anual_dolares",
        "bono_anual": "bono_anual_euros"
    }
```

In [9]:
# escribe tu codigo aqui
df_salarios = df_salarios.rename(
    columns={
        "sueldo_anual": "sueldo_anual_dolares",
        "bono_anual": "bono_anual_euros"
    }
)

Revisar si las columnas cambiaron de nombre con `.columns`

In [10]:
# escribe tu codigo aqui
df_salarios.columns

Index(['nombre', 'apellido', 'edad', 'pais', 'educacion_universitaria',
       'nombre_de_la_empresa', 'cargo', 'anos_en_la_empresa',
       'sueldo_anual_dolares', 'bono_anual_euros'],
      dtype='object')

Reemplazar el simbolo de dólar `$` por  `''` utilizando el método `.replace()` 

### ✂️ Eliminar símbolo de dólar
`str.replace('$','')` quita el carácter pero **deja las comas**.
Aún es tipo `object` (string); falta:
1. eliminar comas,
2. convertir a `float`.

In [11]:
# escribe tu codigo aqui
df_salarios['sueldo_anual_dolares'] = (df_salarios['sueldo_anual_dolares'].str.replace('$', ''))

Cambiar el tipo de dato de sueldo_anual_dolares con `.astype(float)`

> **Nota:** se produce un `ValueError` porque todavía hay **celdas vacías**
  (`''`) o valores con comas que impiden la conversión directa a `float`.

In [12]:

df_salarios['sueldo_anual_dolares'] = df_salarios['sueldo_anual_dolares'].astype(float)

ValueError: could not convert string to float: ''

### Funcionó? o Cuál es el error?

##### Tu respuesta aqui

reemplazar las celdas vacias `''`

### 🛠️ Reemplazar strings vacíos por `NaN`
Usamos `.replace('', np.nan)` para que pandas los reconozca como faltantes
y permita la conversión numérica en el siguiente paso.

In [13]:
# escribe tu codigo aqui
df_salarios['sueldo_anual_dolares'] = (df_salarios['sueldo_anual_dolares'].replace('', np.nan))

Cambiar el tipo de datos con `.astype(float)`

### 🔄 Convertir a numérico
Ahora que no hay símbolos ni strings vacíos, `astype(float)` funciona.
El resultado es una columna `float64` lista para cálculos.

In [14]:
# escribe tu codigo aqui
df_salarios['sueldo_anual_dolares'] = df_salarios['sueldo_anual_dolares'].astype(float)

Revisar que valores de `sueldo_anual_dolares` tienen el valor `np.nan`

### 📌 Ver cuántos salarios quedaron como `NaN`
Esto nos indica cuántos registros tendrán que imputarse
o eliminarse en el siguiente bloque de limpieza.

In [15]:
# escribe tu codigo aqui
df_salarios['sueldo_anual_dolares'].isna().sum()

np.int64(7)

### 🔍 Inspeccionar filas con salario faltante
Listamos las 7 filas donde `sueldo_anual_dolares` es `NaN`.
Podremos decidir si:
* imputar con la mediana por cargo/país,  
* o eliminar si los valores son críticos.

In [16]:
df_salarios[df_salarios['sueldo_anual_dolares'].isna()]

,nombre,apellido,edad,pais,educacion_universitaria,nombre_de_la_empresa,cargo,anos_en_la_empresa,sueldo_anual_dolares,bono_anual_euros
302,María,Ríos,27,Argentina,Sí,Globant,Python Developer,4,NaN,€11287
305,Paula,Díaz,53,Costa Rica,No,NaN,Machine Learning Engineer,10,NaN,€9874
306,Luis,Pérez,27,Chile,No,Facebook,AI Software Engineer,10,NaN,€10379
307,Andrea,Rodríguez,36,Panamá,Sí,Facebook,Project Manager,1,NaN,€3878
308,Juan,Morales,49,Costa Rica,No,Amazon,Software Engineer,2,NaN,€12223
309,Camila,Torres,45,México,No,Facebook,Data Engineer,3,NaN,€2816
310,Diego,Díaz,41,México,No,Google,Data Analyst,0,NaN,€5934


Ahora el mismo proceso con el símbolo de euros `€`

In [17]:
df_salarios['bono_anual_euros'] = (df_salarios['bono_anual_euros'].str.replace('€', ''))

In [18]:
df_salarios['bono_anual_euros'] = (df_salarios['bono_anual_euros'].astype(float))

In [19]:
df_salarios[df_salarios['bono_anual_euros'].isna()]

,nombre,apellido,edad,pais,educacion_universitaria,nombre_de_la_empresa,cargo,anos_en_la_empresa,sueldo_anual_dolares,bono_anual_euros


In [20]:
df_salarios[['sueldo_anual_dolares', 'bono_anual_euros']].head()

,sueldo_anual_dolares,bono_anual_euros
0,71841.0,4981.0
1,142115.0,1252.0
2,92278.0,5262.0
3,97130.0,3671.0
4,61413.0,10190.0


---
## 4 · Detección y manejo de valores nulos
### 4.1 Contar nulos por columna


In [21]:
df_salarios.isnull().sum()

nombre                     0
apellido                   0
edad                       0
pais                       1
educacion_universitaria    0
nombre_de_la_empresa       8
cargo                      0
anos_en_la_empresa         0
sueldo_anual_dolares       7
bono_anual_euros           0
dtype: int64

> **Estrategias comunes:**
- **Eliminar** filas/columnas con muchos nulos (`dropna`).
- **Imputar** con media/mediana/moda (`fillna`).
- **Revisar** si los nulos tienen significado en tu análisis.


Revisemos el caso de la columna `['pais']`

In [22]:
df_salarios[df_salarios["pais"].isna()]

,nombre,apellido,edad,pais,educacion_universitaria,nombre_de_la_empresa,cargo,anos_en_la_empresa,sueldo_anual_dolares,bono_anual_euros
300,Ana,Pérez,51,NaN,No,Oracle,Project Manager,2,67999.0,12035.0


In [23]:
df_salarios = df_salarios.dropna(subset=["pais"])

In [24]:
df_salarios.isnull().sum()

nombre                     0
apellido                   0
edad                       0
pais                       0
educacion_universitaria    0
nombre_de_la_empresa       8
cargo                      0
anos_en_la_empresa         0
sueldo_anual_dolares       7
bono_anual_euros           0
dtype: int64

Ahora revisemos el caso de la columna `["nombre_de_la_empresa"]`

In [25]:
df_salarios[df_salarios["nombre_de_la_empresa"].isna()]

,nombre,apellido,edad,pais,educacion_universitaria,nombre_de_la_empresa,cargo,anos_en_la_empresa,sueldo_anual_dolares,bono_anual_euros
303,Valentina,Ríos,50,Chile,No,NaN,Python Developer,8,50138.0,3937.0
304,Pedro,Díaz,59,Colombia,Sí,NaN,AI Software Engineer,9,133052.0,5600.0
305,Paula,Díaz,53,Costa Rica,No,NaN,Machine Learning Engineer,10,NaN,9874.0
314,Luis,Ramírez,60,Argentina,No,NaN,Data Scientist,4,72011.0,6723.0
315,José,Martínez,35,Chile,Sí,NaN,Data Engineer,3,97114.0,1684.0
316,Carlos,Flores,55,Uruguay,Sí,NaN,Machine Learning Engineer,1,104541.0,3128.0
317,Lucía,Torres,36,Costa Rica,No,NaN,Software Engineer,0,86042.0,8746.0
318,Lucía,González,56,Uruguay,No,NaN,Data Scientist,7,96078.0,11419.0


No es necesario eliminar esos datos, pues podemos clasificarlos como `"Empresa Desconocida"`

In [26]:
df_salarios["nombre_de_la_empresa"] = (df_salarios["nombre_de_la_empresa"].fillna("Empresa Desconocida"))


In [27]:
df_salarios[df_salarios["nombre_de_la_empresa"].isna()]

,nombre,apellido,edad,pais,educacion_universitaria,nombre_de_la_empresa,cargo,anos_en_la_empresa,sueldo_anual_dolares,bono_anual_euros


In [28]:
df_salarios.isnull().sum()

nombre                     0
apellido                   0
edad                       0
pais                       0
educacion_universitaria    0
nombre_de_la_empresa       0
cargo                      0
anos_en_la_empresa         0
sueldo_anual_dolares       7
bono_anual_euros           0
dtype: int64

Revisemos el caso de `["sueldo_anual_dolares"]`

In [29]:
df_salarios[df_salarios["sueldo_anual_dolares"].isna()]

,nombre,apellido,edad,pais,educacion_universitaria,nombre_de_la_empresa,cargo,anos_en_la_empresa,sueldo_anual_dolares,bono_anual_euros
302,María,Ríos,27,Argentina,Sí,Globant,Python Developer,4,NaN,11287.0
305,Paula,Díaz,53,Costa Rica,No,Empresa Desconocida,Machine Learning Engineer,10,NaN,9874.0
306,Luis,Pérez,27,Chile,No,Facebook,AI Software Engineer,10,NaN,10379.0
307,Andrea,Rodríguez,36,Panamá,Sí,Facebook,Project Manager,1,NaN,3878.0
308,Juan,Morales,49,Costa Rica,No,Amazon,Software Engineer,2,NaN,12223.0
309,Camila,Torres,45,México,No,Facebook,Data Engineer,3,NaN,2816.0
310,Diego,Díaz,41,México,No,Google,Data Analyst,0,NaN,5934.0


Al no existir el registro de sueldos, la decision que tomamos fue eliminar esas filas con `dropna()`

In [30]:
df_salarios = df_salarios.dropna(subset=["sueldo_anual_dolares"])

In [31]:
df_salarios.isnull().sum()

nombre                     0
apellido                   0
edad                       0
pais                       0
educacion_universitaria    0
nombre_de_la_empresa       0
cargo                      0
anos_en_la_empresa         0
sueldo_anual_dolares       0
bono_anual_euros           0
dtype: int64

Ahora no tenemos valores nulos

---
## 5 · Detección y manejo de duplicados
Buscamos registros exactos repetidos.


In [32]:
duplicados = df_salarios.duplicated()
print(f'Duplicados encontrados: {duplicados.sum()}')

Duplicados encontrados: 10


In [33]:
df_salarios = df_salarios.drop_duplicates()

In [34]:
duplicados = df_salarios.duplicated()
print(f'Duplicados encontrados: {duplicados.sum()}')

Duplicados encontrados: 0


In [35]:
df_salarios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 307 entries, 0 to 318
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   nombre                   307 non-null    object 
 1   apellido                 307 non-null    object 
 2   edad                     307 non-null    int64  
 3   pais                     307 non-null    object 
 4   educacion_universitaria  307 non-null    object 
 5   nombre_de_la_empresa     307 non-null    object 
 6   cargo                    307 non-null    object 
 7   anos_en_la_empresa       307 non-null    int64  
 8   sueldo_anual_dolares     307 non-null    float64
 9   bono_anual_euros         307 non-null    float64
dtypes: float64(2), int64(2), object(6)
memory usage: 26.4+ KB


### 🔢 Reiniciar índices
`reset_index(drop=True)` crea un nuevo índice consecutivo (0 → n-1) y descarta el índice anterior.  
Útil después de eliminar filas o reordenar el DataFrame para evitar huecos o duplicados en la numeración.

In [36]:
df_salarios = df_salarios.reset_index(drop=True)

In [37]:
df_salarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307 entries, 0 to 306
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   nombre                   307 non-null    object 
 1   apellido                 307 non-null    object 
 2   edad                     307 non-null    int64  
 3   pais                     307 non-null    object 
 4   educacion_universitaria  307 non-null    object 
 5   nombre_de_la_empresa     307 non-null    object 
 6   cargo                    307 non-null    object 
 7   anos_en_la_empresa       307 non-null    int64  
 8   sueldo_anual_dolares     307 non-null    float64
 9   bono_anual_euros         307 non-null    float64
dtypes: float64(2), int64(2), object(6)
memory usage: 24.1+ KB


### 💱 Conversión de Euros a Dólares  
Definimos una **tasa de cambio fija** `TASA_EUR_USD = 1.10` (1 € → 1.10 US$)  
y creamos la columna `bono_anual_dolares` multiplicando cada valor en  
`bono_anual_euros` por esa tasa. Esta operación es vectorizada: se aplica  
a toda la columna sin necesidad de bucles y deja ambos montos disponibles  
para comparaciones o visualizaciones futuras.

In [38]:
df_salarios.head()

,nombre,apellido,edad,pais,educacion_universitaria,nombre_de_la_empresa,cargo,anos_en_la_empresa,sueldo_anual_dolares,bono_anual_euros
0,José,López,36,Chile,Sí,Mercado Libre,Python Developer,2,71841.0,4981.0
1,Fernanda,Vargas,39,Chile,Sí,Amazon,AI Software Engineer,2,142115.0,1252.0
2,Fernanda,Ramírez,30,Colombia,No,Facebook,Machine Learning Engineer,2,92278.0,5262.0
3,José,Gómez,35,Chile,Sí,Mercado Libre,Python Developer,12,97130.0,3671.0
4,Ana,Pérez,32,Panamá,No,Amazon,Project Manager,2,61413.0,10190.0


In [39]:
# 1 € ≈ 1.10 USD
TASA_EUR_USD = 1.10

In [40]:
df_salarios["bono_anual_dolares"] = (
    df_salarios["bono_anual_euros"] * TASA_EUR_USD
)

In [41]:
df_salarios[["bono_anual_euros", "bono_anual_dolares"]]

,bono_anual_euros,bono_anual_dolares
0,4981.0,5479.1
1,1252.0,1377.2
2,5262.0,5788.2
3,3671.0,4038.1
4,10190.0,11209.0
...,...,...
302,6723.0,7395.3
303,1684.0,1852.4
304,3128.0,3440.8
305,8746.0,9620.6


### 📏 Ajuste de precisión  
Redondeamos `bono_anual_dolares` a **una cifra decimal** (`round(1)`) para  
homogeneizar el formato de los valores monetarios y evitar “ruido” de  
centavos insignificantes al presentar o agrupar los datos.

In [42]:
df_salarios["bono_anual_dolares"] = df_salarios["bono_anual_dolares"].round(1)

In [43]:
df_salarios[["bono_anual_euros", "bono_anual_dolares"]]

,bono_anual_euros,bono_anual_dolares
0,4981.0,5479.1
1,1252.0,1377.2
2,5262.0,5788.2
3,3671.0,4038.1
4,10190.0,11209.0
...,...,...
302,6723.0,7395.3
303,1684.0,1852.4
304,3128.0,3440.8
305,8746.0,9620.6


In [44]:
df_salarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307 entries, 0 to 306
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   nombre                   307 non-null    object 
 1   apellido                 307 non-null    object 
 2   edad                     307 non-null    int64  
 3   pais                     307 non-null    object 
 4   educacion_universitaria  307 non-null    object 
 5   nombre_de_la_empresa     307 non-null    object 
 6   cargo                    307 non-null    object 
 7   anos_en_la_empresa       307 non-null    int64  
 8   sueldo_anual_dolares     307 non-null    float64
 9   bono_anual_euros         307 non-null    float64
 10  bono_anual_dolares       307 non-null    float64
dtypes: float64(3), int64(2), object(6)
memory usage: 26.5+ KB


## 6 · Cargar el dataframe de países


In [45]:
#definir un path o ruta con Path
path_paises = DATA_RAW_DIR / "paises.csv"
path_paises = str(path_paises)

print(f"Ruta del archivo paises: {path_paises}")
print(f"existe archivo paises: {Path(path_paises).exists()}")

Ruta del archivo paises: /Users/jssdev/Dev/Projects/Pydata/pydatapanama-cursos/cursos/analisis_datos/data/raw/paises.csv
existe archivo paises: True


In [46]:
df_paises = pd.read_csv(filepath_or_buffer=path_paises, sep=',', encoding='utf-8')

In [47]:
df_paises.head()

,pais,capital,cantidad_de_habitantes,PIB,ingreso_per_capita
0,Argentina,Buenos Aires,45376763,"$641,000,000,000","$14,120"
1,Brasil,Brasilia,214326223,"$2,000,000,000,000","$9,334"
2,Chile,Santiago,19116209,"$317,000,000,000","$16,580"
3,Colombia,Bogotá,51874024,"$343,000,000,000","$6,611"
4,México,Ciudad de México,126014024,"$1,410,000,000,000","$11,190"


In [48]:
df_paises['pais'].unique()

array(['Argentina', 'Brasil', 'Chile', 'Colombia', 'México', 'Perú',
       'Uruguay', 'Panamá', 'Ecuador', 'Bolivia'], dtype=object)

In [49]:
df_paises['pais'].nunique()

10

In [50]:
df_salarios['pais'].unique()

array(['Chile', 'Colombia', 'Panamá', 'Costa Rica', 'México', 'Uruguay',
       'Argentina', 'Perú'], dtype=object)

In [51]:
df_salarios['pais'].nunique()

8

In [52]:
paises_en_df_paises = set(df_paises["pais"])
paises_en_df_paises

{'Argentina',
 'Bolivia',
 'Brasil',
 'Chile',
 'Colombia',
 'Ecuador',
 'México',
 'Panamá',
 'Perú',
 'Uruguay'}

In [53]:
paises_en_df_salarios = set(df_salarios["pais"])
paises_en_df_salarios

{'Argentina',
 'Chile',
 'Colombia',
 'Costa Rica',
 'México',
 'Panamá',
 'Perú',
 'Uruguay'}

### 🔍 Identificar discrepancias entre DataFrames

1. **`paises_faltantes_en_df_paises`**  
   Conjunto de países que **están en `df_paises` pero no aparecen** en `df_salarios`.

2. **`paises_faltantes_en_df_salarios`**  
   Conjunto inverso: países presentes en `df_salarios`, ausentes en `df_paises`.

3. **`paises_faltantes`**  
   Unión (`|`) de ambos conjuntos → lista global de **todos los países que no coinciden** entre los dos DataFrames.  
   Útil para decidir si imputar, descartar o investigar la fuente de datos.

In [54]:
paises_faltantes_en_df_paises = paises_en_df_paises - paises_en_df_salarios
paises_faltantes_en_df_paises

{'Bolivia', 'Brasil', 'Ecuador'}

In [55]:
paises_faltantes_en_df_salarios =  paises_en_df_salarios - paises_en_df_paises
paises_faltantes_en_df_salarios

{'Costa Rica'}

In [56]:
paises_faltantes = paises_faltantes_en_df_paises | paises_faltantes_en_df_salarios
paises_faltantes

{'Bolivia', 'Brasil', 'Costa Rica', 'Ecuador'}

In [57]:
df_paises[df_paises['pais'].isin(paises_faltantes)]

,pais,capital,cantidad_de_habitantes,PIB,ingreso_per_capita
1,Brasil,Brasilia,214326223,"$2,000,000,000,000","$9,334"
8,Ecuador,Quito,18001000,"$118,000,000,000","$6,556"
9,Bolivia,La Paz,12456000,"$44,000,000,000","$3,530"


### 🗑️ Eliminar registros específicos  
Usamos `drop(index=[1, 8, 9])` para remover las filas cuyo índice es 1, 8 y 9  
(Brasil, Ecuador y Bolivia en este caso). Esto excluye esos países del  
DataFrame `df_paises` antes de realizar uniones o análisis posteriores.

In [58]:
df_paises = df_paises.drop(index=[1, 8, 9])

In [59]:
df_paises.reset_index(drop=True, inplace=True)

In [60]:
df_paises

,pais,capital,cantidad_de_habitantes,PIB,ingreso_per_capita
0,Argentina,Buenos Aires,45376763,"$641,000,000,000","$14,120"
1,Chile,Santiago,19116209,"$317,000,000,000","$16,580"
2,Colombia,Bogotá,51874024,"$343,000,000,000","$6,611"
3,México,Ciudad de México,126014024,"$1,410,000,000,000","$11,190"
4,Perú,Lima,33925854,"$268,000,000,000","$7,900"
5,Uruguay,Montevideo,3518552,"$71,000,000,000","$20,300"
6,Panamá,Ciudad de Panamá,4468000,"$76,000,000,000","$16,995"


### 💲 Limpieza y conversión de columnas monetarias  
Seleccionamos las columnas `PIB` e `ingreso_per_capita` y:

1. **`replace({'\\$': '', ',': ''}, regex=True)`**  
   - Elimina el símbolo de dólar (`$`) y las comas de miles (`,`).  
   - `regex=True` permite usar patrones de expresión regular.

2. **`astype(float)`**  
   - Convierte las cadenas ya limpias a valores numéricos (`float64`).

Resultado: ambas columnas quedan listas para cálculos y comparaciones
estadísticas sin símbolos ni separadores.

In [61]:
df_paises[["PIB", "ingreso_per_capita"]] = (df_paises[["PIB", "ingreso_per_capita"]].replace({'\\$': '', ',': ''}, regex=True).astype(float))

In [62]:
df_paises.head()

,pais,capital,cantidad_de_habitantes,PIB,ingreso_per_capita
0,Argentina,Buenos Aires,45376763,6.410000e+11,14120.0
1,Chile,Santiago,19116209,3.170000e+11,16580.0
2,Colombia,Bogotá,51874024,3.430000e+11,6611.0
3,México,Ciudad de México,126014024,1.410000e+12,11190.0
4,Perú,Lima,33925854,2.680000e+11,7900.0


In [63]:
df_paises.isnull().sum()

pais                      0
capital                   0
cantidad_de_habitantes    0
PIB                       0
ingreso_per_capita        0
dtype: int64

---
## 7 · Unir datasets (`merge`)
Unimos `df_salarios` con `df_paises` por la columna `pais` para enriquecer la información.


In [64]:
df_salarios_pais = pd.merge(df_salarios, df_paises, on='pais', how='left').reset_index(drop=True)


In [65]:
df_salarios_pais.head()

,nombre,apellido,edad,pais,educacion_universitaria,nombre_de_la_empresa,cargo,anos_en_la_empresa,sueldo_anual_dolares,bono_anual_euros,bono_anual_dolares,capital,cantidad_de_habitantes,PIB,ingreso_per_capita
0,José,López,36,Chile,Sí,Mercado Libre,Python Developer,2,71841.0,4981.0,5479.1,Santiago,19116209.0,3.170000e+11,16580.0
1,Fernanda,Vargas,39,Chile,Sí,Amazon,AI Software Engineer,2,142115.0,1252.0,1377.2,Santiago,19116209.0,3.170000e+11,16580.0
2,Fernanda,Ramírez,30,Colombia,No,Facebook,Machine Learning Engineer,2,92278.0,5262.0,5788.2,Bogotá,51874024.0,3.430000e+11,6611.0
3,José,Gómez,35,Chile,Sí,Mercado Libre,Python Developer,12,97130.0,3671.0,4038.1,Santiago,19116209.0,3.170000e+11,16580.0
4,Ana,Pérez,32,Panamá,No,Amazon,Project Manager,2,61413.0,10190.0,11209.0,Ciudad de Panamá,4468000.0,7.600000e+10,16995.0


---
## 8 · Crear columnas derivadas


### 💰 Métrica de compensación total  
Creamos `compensacion_total` sumando el salario anual (`sueldo_anual_dolares`)  
y el bono anual convertido a dólares (`bono_anual_dolares`).  
Esta columna refleja cuánto percibe realmente cada empleado en un año,  
facilitando análisis comparativos por país, cargo o empresa.

In [66]:
df_salarios_pais['compensacion_total'] = df_salarios_pais['sueldo_anual_dolares'] + df_salarios_pais['bono_anual_dolares']


In [67]:
df_salarios_pais.head()

,nombre,apellido,edad,pais,educacion_universitaria,nombre_de_la_empresa,cargo,anos_en_la_empresa,sueldo_anual_dolares,bono_anual_euros,bono_anual_dolares,capital,cantidad_de_habitantes,PIB,ingreso_per_capita,compensacion_total
0,José,López,36,Chile,Sí,Mercado Libre,Python Developer,2,71841.0,4981.0,5479.1,Santiago,19116209.0,3.170000e+11,16580.0,77320.1
1,Fernanda,Vargas,39,Chile,Sí,Amazon,AI Software Engineer,2,142115.0,1252.0,1377.2,Santiago,19116209.0,3.170000e+11,16580.0,143492.2
2,Fernanda,Ramírez,30,Colombia,No,Facebook,Machine Learning Engineer,2,92278.0,5262.0,5788.2,Bogotá,51874024.0,3.430000e+11,6611.0,98066.2
3,José,Gómez,35,Chile,Sí,Mercado Libre,Python Developer,12,97130.0,3671.0,4038.1,Santiago,19116209.0,3.170000e+11,16580.0,101168.1
4,Ana,Pérez,32,Panamá,No,Amazon,Project Manager,2,61413.0,10190.0,11209.0,Ciudad de Panamá,4468000.0,7.600000e+10,16995.0,72622.0


### 🏆 Clasificación de senioridad  
Creamos la columna `senioridad` con `np.where`:

* **Condición** → `anos_en_la_empresa > 5`  
  - Si el empleado lleva **más de 5 años**, se le etiqueta como **"Senior"**.  
  - En caso contrario, se marca como **"Junior"**.

Así obtenemos una variable categórica que nos ayudará a comparar
compensaciones y otros indicadores entre niveles de experiencia.

In [68]:
df_salarios_pais['senioridad'] = np.where(df_salarios_pais['anos_en_la_empresa'] > 5, 'Senior', 'Junior')


In [69]:
df_salarios_pais[['nombre', 'apellido', 'anos_en_la_empresa','compensacion_total', 'senioridad']].head(10)

,nombre,apellido,anos_en_la_empresa,compensacion_total,senioridad
0,José,López,2,77320.1,Junior
1,Fernanda,Vargas,2,143492.2,Junior
2,Fernanda,Ramírez,2,98066.2,Junior
3,José,Gómez,12,101168.1,Senior
4,Ana,Pérez,2,72622.0,Junior
5,Andrea,Torres,8,91029.3,Senior
6,Andrea,Vargas,9,102338.1,Senior
7,Pedro,Herrera,1,164982.1,Junior
8,Juan,Torres,9,98870.5,Senior
9,María,Vargas,13,118492.4,Senior


### 🔎 Revisión de valores nulos por columna  
`df_salarios_pais.isna().sum()` muestra cuántos `NaN` hay en cada columna  
tras el merge. Los resultados indican que los registros de **Costa Rica**  
aún presentan nulos en `capital`, `PIB`, `cantidad_de_habitantes` e  
`ingreso_per_capita`. Conviene imputar o investigar estos campos antes de  
continuar con el análisis para evitar sesgos o errores en cálculos agregados.

In [70]:
df_salarios_pais.isna().sum()

nombre                      0
apellido                    0
edad                        0
pais                        0
educacion_universitaria     0
nombre_de_la_empresa        0
cargo                       0
anos_en_la_empresa          0
sueldo_anual_dolares        0
bono_anual_euros            0
bono_anual_dolares          0
capital                    39
cantidad_de_habitantes     39
PIB                        39
ingreso_per_capita         39
compensacion_total          0
senioridad                  0
dtype: int64

In [71]:
df_salarios_pais[df_salarios_pais['pais'] == 'Costa Rica']

,nombre,apellido,edad,pais,educacion_universitaria,nombre_de_la_empresa,cargo,anos_en_la_empresa,sueldo_anual_dolares,bono_anual_euros,bono_anual_dolares,capital,cantidad_de_habitantes,PIB,ingreso_per_capita,compensacion_total,senioridad
5,Andrea,Torres,45,Costa Rica,No,Mercado Libre,Data Engineer,8,84987.0,5493.0,6042.3,NaN,NaN,NaN,NaN,91029.3,Senior
16,Fernanda,Ríos,23,Costa Rica,Sí,Oracle,Data Engineer,10,97616.0,8345.0,9179.5,NaN,NaN,NaN,NaN,106795.5,Senior
21,Pedro,Ramírez,54,Costa Rica,Sí,Microsoft,Machine Learning Engineer,2,101737.0,9828.0,10810.8,NaN,NaN,NaN,NaN,112547.8,Junior
23,Pedro,Torres,22,Costa Rica,Sí,Facebook,AI Software Engineer,6,115269.0,10657.0,11722.7,NaN,NaN,NaN,NaN,126991.7,Senior
32,Carlos,Herrera,53,Costa Rica,No,Rappi,Data Scientist,4,79861.0,7554.0,8309.4,NaN,NaN,NaN,NaN,88170.4,Junior
34,Fernanda,Díaz,38,Costa Rica,No,Oracle,Project Manager,0,89704.0,7063.0,7769.3,NaN,NaN,NaN,NaN,97473.3,Junior
35,Diego,González,40,Costa Rica,No,Globant,Machine Learning Engineer,12,104739.0,11879.0,13066.9,NaN,NaN,NaN,NaN,117805.9,Senior
41,Lucía,López,49,Costa Rica,Sí,Meta,Data Scientist,13,61352.0,7054.0,7759.4,NaN,NaN,NaN,NaN,69111.4,Senior
45,Pedro,Vargas,28,Costa Rica,No,Facebook,Data Analyst,8,56598.0,3690.0,4059.0,NaN,NaN,NaN,NaN,60657.0,Senior
51,Fernanda,Torres,47,Costa Rica,Sí,Google,Product Owner,7,87181.0,7291.0,8020.1,NaN,NaN,NaN,NaN,95201.1,Senior


### 🔍 Filtrar filas de Costa Rica  
Creamos `mask_cr` para identificar las filas donde `pais == "Costa Rica"` y,  
con `loc`, mostramos únicamente esos registros. Así podemos inspeccionar  
qué columnas siguen con `NaN` y decidir cómo imputarlas o ajustarlas.

In [72]:
mask_cr = df_salarios_pais["pais"] == "Costa Rica"
df_salarios_pais.loc[mask_cr]

,nombre,apellido,edad,pais,educacion_universitaria,nombre_de_la_empresa,cargo,anos_en_la_empresa,sueldo_anual_dolares,bono_anual_euros,bono_anual_dolares,capital,cantidad_de_habitantes,PIB,ingreso_per_capita,compensacion_total,senioridad
5,Andrea,Torres,45,Costa Rica,No,Mercado Libre,Data Engineer,8,84987.0,5493.0,6042.3,NaN,NaN,NaN,NaN,91029.3,Senior
16,Fernanda,Ríos,23,Costa Rica,Sí,Oracle,Data Engineer,10,97616.0,8345.0,9179.5,NaN,NaN,NaN,NaN,106795.5,Senior
21,Pedro,Ramírez,54,Costa Rica,Sí,Microsoft,Machine Learning Engineer,2,101737.0,9828.0,10810.8,NaN,NaN,NaN,NaN,112547.8,Junior
23,Pedro,Torres,22,Costa Rica,Sí,Facebook,AI Software Engineer,6,115269.0,10657.0,11722.7,NaN,NaN,NaN,NaN,126991.7,Senior
32,Carlos,Herrera,53,Costa Rica,No,Rappi,Data Scientist,4,79861.0,7554.0,8309.4,NaN,NaN,NaN,NaN,88170.4,Junior
34,Fernanda,Díaz,38,Costa Rica,No,Oracle,Project Manager,0,89704.0,7063.0,7769.3,NaN,NaN,NaN,NaN,97473.3,Junior
35,Diego,González,40,Costa Rica,No,Globant,Machine Learning Engineer,12,104739.0,11879.0,13066.9,NaN,NaN,NaN,NaN,117805.9,Senior
41,Lucía,López,49,Costa Rica,Sí,Meta,Data Scientist,13,61352.0,7054.0,7759.4,NaN,NaN,NaN,NaN,69111.4,Senior
45,Pedro,Vargas,28,Costa Rica,No,Facebook,Data Analyst,8,56598.0,3690.0,4059.0,NaN,NaN,NaN,NaN,60657.0,Senior
51,Fernanda,Torres,47,Costa Rica,Sí,Google,Product Owner,7,87181.0,7291.0,8020.1,NaN,NaN,NaN,NaN,95201.1,Senior


### 📝 Diccionario con datos oficiales de Costa Rica  
Definimos `datos_cr` con los valores verificados para imputar en las filas  
de Costa Rica:

* **capital:** San José  
* **cantidad_de_habitantes:** 5 150 000  
* **PIB:** 86 500 000 000 USD  
* **ingreso_per_capita:** 14 319 USD

Estos números reemplazarán los `NaN` correspondientes en el DataFrame.

In [73]:
datos_cr = {
    "capital": "San José",
    "cantidad_de_habitantes": 5150000,
    "PIB": 86500000000,
    "ingreso_per_capita": 14319
}

### 🛠️ Imputar valores faltantes para Costa Rica  
Recorremos `datos_cr` y, para cada columna:

1. **Ubicamos** únicamente las filas de Costa Rica (`mask_cr`).  
2. **`fillna(valor)`** reemplaza solo los `NaN` con el dato oficial correspondiente.  
3. Asignamos el resultado de vuelta a la misma columna, dejando intactos los
   registros de otros países.

De este modo, Costa Rica queda completa sin alterar los valores ya correctos.

In [74]:
for col, valor in datos_cr.items():
    df_salarios_pais.loc[mask_cr, col] = (df_salarios_pais.loc[mask_cr, col].fillna(valor))

In [75]:
df_salarios_pais.loc[mask_cr]

,nombre,apellido,edad,pais,educacion_universitaria,nombre_de_la_empresa,cargo,anos_en_la_empresa,sueldo_anual_dolares,bono_anual_euros,bono_anual_dolares,capital,cantidad_de_habitantes,PIB,ingreso_per_capita,compensacion_total,senioridad
5,Andrea,Torres,45,Costa Rica,No,Mercado Libre,Data Engineer,8,84987.0,5493.0,6042.3,San José,5150000.0,8.650000e+10,14319.0,91029.3,Senior
16,Fernanda,Ríos,23,Costa Rica,Sí,Oracle,Data Engineer,10,97616.0,8345.0,9179.5,San José,5150000.0,8.650000e+10,14319.0,106795.5,Senior
21,Pedro,Ramírez,54,Costa Rica,Sí,Microsoft,Machine Learning Engineer,2,101737.0,9828.0,10810.8,San José,5150000.0,8.650000e+10,14319.0,112547.8,Junior
23,Pedro,Torres,22,Costa Rica,Sí,Facebook,AI Software Engineer,6,115269.0,10657.0,11722.7,San José,5150000.0,8.650000e+10,14319.0,126991.7,Senior
32,Carlos,Herrera,53,Costa Rica,No,Rappi,Data Scientist,4,79861.0,7554.0,8309.4,San José,5150000.0,8.650000e+10,14319.0,88170.4,Junior
34,Fernanda,Díaz,38,Costa Rica,No,Oracle,Project Manager,0,89704.0,7063.0,7769.3,San José,5150000.0,8.650000e+10,14319.0,97473.3,Junior
35,Diego,González,40,Costa Rica,No,Globant,Machine Learning Engineer,12,104739.0,11879.0,13066.9,San José,5150000.0,8.650000e+10,14319.0,117805.9,Senior
41,Lucía,López,49,Costa Rica,Sí,Meta,Data Scientist,13,61352.0,7054.0,7759.4,San José,5150000.0,8.650000e+10,14319.0,69111.4,Senior
45,Pedro,Vargas,28,Costa Rica,No,Facebook,Data Analyst,8,56598.0,3690.0,4059.0,San José,5150000.0,8.650000e+10,14319.0,60657.0,Senior
51,Fernanda,Torres,47,Costa Rica,Sí,Google,Product Owner,7,87181.0,7291.0,8020.1,San José,5150000.0,8.650000e+10,14319.0,95201.1,Senior


### ✅ Verificación final de nulos  
Ejecutamos `df_salarios_pais.isnull().sum()` para confirmar que **todas las  
columnas ahora muestran 0 valores faltantes**. Nuestro DataFrame está listo  
para pasar al Análisis Exploratorio de Datos sin riesgos de errores por `NaN`.

In [76]:
df_salarios_pais.isnull().sum()

nombre                     0
apellido                   0
edad                       0
pais                       0
educacion_universitaria    0
nombre_de_la_empresa       0
cargo                      0
anos_en_la_empresa         0
sueldo_anual_dolares       0
bono_anual_euros           0
bono_anual_dolares         0
capital                    0
cantidad_de_habitantes     0
PIB                        0
ingreso_per_capita         0
compensacion_total         0
senioridad                 0
dtype: int64

In [77]:
df_salarios_pais.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307 entries, 0 to 306
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   nombre                   307 non-null    object 
 1   apellido                 307 non-null    object 
 2   edad                     307 non-null    int64  
 3   pais                     307 non-null    object 
 4   educacion_universitaria  307 non-null    object 
 5   nombre_de_la_empresa     307 non-null    object 
 6   cargo                    307 non-null    object 
 7   anos_en_la_empresa       307 non-null    int64  
 8   sueldo_anual_dolares     307 non-null    float64
 9   bono_anual_euros         307 non-null    float64
 10  bono_anual_dolares       307 non-null    float64
 11  capital                  307 non-null    object 
 12  cantidad_de_habitantes   307 non-null    float64
 13  PIB                      307 non-null    float64
 14  ingreso_per_capita       3

---
## 9 · Exportar dataset limpio


In [78]:
from cursos.analisis_datos.utils.paths import DATA_PROCESSED_DIR

In [79]:
salarios_pais_path = DATA_PROCESSED_DIR / "salarios_pais.csv"
salarios_pais_path = str(salarios_pais_path)

print(f"Ruta del archivo salarios_paises: {salarios_pais_path}")
print(f"existe archivo salarios_pais: {Path(salarios_pais_path).exists()}")

Ruta del archivo salarios_paises: /Users/jssdev/Dev/Projects/Pydata/pydatapanama-cursos/cursos/analisis_datos/data/processed/salarios_pais.csv
existe archivo salarios_pais: False


### 💾 Exportar dataset limpio  
`to_csv(..., index=False, sep=',', encoding='utf-8')` guarda `df_salarios_pais` como un CSV en la ruta definida por `salarios_pais_path`, sin incluir la columna de índice y usando codificación UTF-8.  
La línea `print()` confirma en pantalla la ubicación del archivo generado, facilitando su localización para futuros análisis o compartición.

In [80]:
df_salarios_pais.to_csv(path_or_buf=salarios_pais_path, index=False, sep=',', encoding='utf-8')


In [81]:
print(f' Dataset limpio exportado a {salarios_pais_path}')

 Dataset limpio exportado a /Users/jssdev/Dev/Projects/Pydata/pydatapanama-cursos/cursos/analisis_datos/data/processed/salarios_pais.csv


In [82]:
print(f"existe archivo salarios_pais: {Path(salarios_pais_path).exists()}")

existe archivo salarios_pais: True


---
## 10 · Resumen 
En esta práctica aprendiste a:
- Limpiar columnas con símbolos.
- Detectar/llenar nulos y eliminar duplicados.
- Realizar un **merge** y enriquecer tu información.
- Crear métricas derivadas y exportar datos limpios.

